# **Processing Data**
### Mục tiêu
* **Làm sạch dữ liệu:** Loại bỏ nhiễu, các giá trị trùng lặp và các tin đăng không thực tế (outliers thô).
* **Chuẩn hóa nội dung:** Đồng nhất định dạng văn bản, địa chỉ và các đơn vị đo lường.
* **Feature Engineering:** 
    * Trích xuất tọa độ địa lý (Lat/Long) và tính khoảng cách đến trung tâm (Quận 1).
    * Xây dựng các chỉ số mới như tỷ lệ tiện ích (`amenity_ratio`) và xác định các tuyến đường trọng điểm (`is_hot_street`).
* **Kiểm soát biến số:** Xử lý giá trị ngoại lai bằng phương pháp IQR để đảm bảo tính ổn định cho mô hình.

## **Import thư viện**

In [1]:
import sys
from tqdm import tqdm
import numpy as np
import pandas as pd

sys.path.append("../utilities")

from processing import (
    # text
    clean_text, is_nfc, normalize_vn_title,

    # geocode
    setup_geocoder, get_lat_lon, build_area_nominatim,
    save_cache, load_cache, haversine_km,

    # features / FE
    extract_district_from_address, add_district_median_price,
    add_hot_street_by_relative_price,

    # outliers / audit
    iqr_cap, audit_case_variants,
)


## **Load data**

In [2]:
df = pd.read_csv("../Data/cleaned.csv")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24121 entries, 0 to 24120
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             24121 non-null  object 
 1   description       24113 non-null  object 
 2   location          24113 non-null  object 
 3   address           24121 non-null  object 
 4   street_name       24120 non-null  object 
 5   price             24062 non-null  float64
 6   area              24113 non-null  float64
 7   date              24113 non-null  object 
 8   air_conditioning  24121 non-null  int64  
 9   fridge            24121 non-null  int64  
 10  washing_machine   24121 non-null  int64  
 11  mezzanine         24121 non-null  int64  
 12  kitchen           24121 non-null  int64  
 13  wardrobe          24121 non-null  int64  
 14  bed               24121 non-null  int64  
 15  balcony           24121 non-null  int64  
 16  elevator          24121 non-null  int64 

## **Xóa các địa bàn không phải ở khu vực TP.HCM**
Thông qua quá trình phân tích và khám phá dữ liệu (Exploratory Data Analysis – EDA), nhận thấy rằng số lượng tin đăng tại khu vực **TP. Hồ Chí Minh chiếm gần 90% tổng số mẫu dữ liệu**. Sự phân bố này cho thấy dữ liệu bị **mất cân bằng nghiêm trọng theo yếu tố địa lý (geographical bias)**.

Việc xây dựng một mô hình dự đoán giá thuê phòng trọ trên phạm vi **toàn quốc** trong bối cảnh dữ liệu bị lệch mạnh về một khu vực cụ thể có thể dẫn đến các vấn đề sau:
- Mô hình học chủ yếu theo mặt bằng giá và đặc trưng của TP. Hồ Chí Minh, trong khi không phản ánh đúng đặc điểm thị trường ở các tỉnh, thành khác.
- Kết quả dự đoán cho các khu vực ngoài TP. Hồ Chí Minh thiếu độ tin cậy và có sai lệch lớn.
- Gia tăng nguy cơ **bias trong mô hình** và làm giảm khả năng tổng quát hóa (generalization).

Do đó, để đảm bảo tính nhất quán của dữ liệu, giảm thiểu sai lệch và nâng cao chất lượng mô hình, bài toán được **giới hạn lại thành bài toán dự đoán giá thuê phòng trọ trong phạm vi TP. Hồ Chí Minh**. Việc giới hạn không gian địa lý này giúp mô hình tập trung học đúng mặt bằng giá, các yếu tố tiện ích và đặc trưng khu vực, từ đó cho kết quả dự đoán chính xác và có ý nghĩa hơn trong thực tế.

In [3]:
n_before = len(df)
print(f"BEFORE: {n_before:,} rows")

# Lọc theo địa chỉ TP.HCM
df = df[df["address"].str.contains("Hồ Chí Minh", na=False)]

n_after = len(df)
print(f"AFTER: {n_after:,} rows")

print(f"REMOVED: {n_before - n_after:,} rows "
      f"({(n_before - n_after) / n_before * 100:.2f}%)")


BEFORE: 24,121 rows
AFTER: 21,077 rows
REMOVED: 3,044 rows (12.62%)


## **Xử lí các dòng trùng lặp & NaN**

### **Xử lí dòng trùng lặp**

In [4]:
print("BEFORE: Duplicate rows:", df.duplicated().sum())
df = df.drop_duplicates()
print("AFTER: Duplicate rows:", df.duplicated().sum())


BEFORE: Duplicate rows: 107
AFTER: Duplicate rows: 0


### **Xử lí dòng NaN**

In [5]:
before = df.shape[0]
print(f"Before: {before} dòng.")

df = df.dropna()

print(f"After : {df.shape[0]} dòng.")
print(f"Đã loại bỏ {before - df.shape[0]} dòng.")


Before: 20970 dòng.
After : 20932 dòng.
Đã loại bỏ 38 dòng.


## **Loại bỏ các cột không cần thiết (Feature Selection)**

Việc loại bỏ các cột như `url`, `description`, `title`, và `location` là một bước quan trọng trong tiền xử lý dữ liệu vì các lý do sau:

* **`url`**: Thường là định danh duy nhất cho mỗi dòng dữ liệu trên web. Nó không mang giá trị thống kê hay giúp mô hình học được xu hướng chung của thị trường.
* **`description` & `title`**: Đây là dữ liệu văn bản không cấu trúc (Unstructured Text). Nếu không sử dụng các kỹ thuật xử lý ngôn ngữ tự nhiên (NLP), các cột này sẽ gây nhiễu và làm tăng dung lượng bộ nhớ không cần thiết.
* **`location`**: Thường chứa dữ liệu dạng địa chỉ chi tiết có độ nhiễu cao (High Cardinality). Do chúng ta đã có các cột đặc trưng khác (như quận, huyện, tên đường ...), cột địa chỉ thô này trở nên dư thừa.

In [6]:
cols_to_drop = ["url", "description", "title", "location"]

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

print("Các cột còn lại:")
print(df.columns.tolist())


Các cột còn lại:
['address', 'street_name', 'price', 'area', 'date', 'air_conditioning', 'fridge', 'washing_machine', 'mezzanine', 'kitchen', 'wardrobe', 'bed', 'balcony', 'elevator', 'free_time', 'parking']


## **Feature Engineering**

### **1. Tiền xử lí**

#### 1.1. Loại bỏ các giá trị lỗi

Trước khi tiến hành **Feature Engineering**, dữ liệu được lọc nhằm loại bỏ các giá trị lỗi hoặc phi thực tế. Việc thiết lập các ngưỡng chặn này là cần thiết để đảm bảo mô hình không bị nhiễu bởi các điểm dữ liệu ngoại lai (outliers) hoặc sai sót trong quá trình nhập liệu.

**Cụ thể:**

* **Về giá thuê (price):** Chỉ giữ lại các bản ghi có giá thuê dương và không vượt quá **200 triệu đồng/tháng**.
    * *Lý do:* Loại bỏ các tin đăng có giá bằng 0 (thông tin không đầy đủ) hoặc các giá trị cực đoan do người dùng nhập thừa số 0.
* **Về diện tích (area):** Chỉ giữ lại các bản ghi có diện tích dương và không vượt quá **300 m²**.
    * *Lý do:* Đảm bảo diện tích phòng phù hợp với thực tế sử dụng của các loại hình phòng trọ, căn hộ và nhà nguyên căn. Các giá trị vượt ngưỡng này gần như là giá trị rác, lỗi... có thể do người đăng tin nhập thừa số '0' chẳng hạn.

In [7]:
# Lưu số lượng bản ghi trước khi lọc
initial_count = len(df)

# Thực hiện lọc các giá trị phi thực tế
df = df[
    (df["price"] > 0) & (df["price"] <= 200) & 
    (df["area"] > 0) & (df["area"] <= 300)
]

# Tính toán số lượng bản ghi bị loại bỏ
removed_count = initial_count - len(df)

print(f"Số lượng bản ghi bị loại bỏ do vi phạm điều kiện: {removed_count}")
print(f"Số lượng bản ghi còn lại sau khi làm sạch: {len(df)}")


Số lượng bản ghi bị loại bỏ do vi phạm điều kiện: 20
Số lượng bản ghi còn lại sau khi làm sạch: 20912


#### 1.2. Chuẩn hóa Unicode & khoảng trắng
Các ký tự tiếng Việt ở dạng non-NFC có thể gây lỗi khi so sánh chuỗi, lọc trùng hoặc phân tích nội dung mô tả. Vì vậy, toàn bộ văn bản được chuẩn hóa về Unicode NFC để đảm bảo biểu diễn thống nhất. Đồng thời, các khoảng trắng dư thừa được rút gọn và loại bỏ ở đầu, cuối chuỗi.

In [8]:
cols = ["address", "street_name"]

print("Before")
for c in cols:
    bad = (~df[c].astype(str).apply(is_nfc)).sum()
    print(f"{c}: {bad} non-NFC rows")


df[cols] = df[cols].apply(lambda s: s.map(clean_text))

print("\nAfter")
for c in cols:
    bad = (~df[c].astype(str).apply(is_nfc)).sum()
    print(f"{c}: {bad} non-NFC rows")


Before
address: 0 non-NFC rows
street_name: 12 non-NFC rows

After
address: 0 non-NFC rows
street_name: 0 non-NFC rows


#### 1.3 Kiểm tra về đồng nhất giá trị

Mục tiêu là phát hiện các giá trị **giống nhau về nội dung** nhưng **khác cách viết hoa – thường** (ví dụ: `Hoàng Hoa Thám` vs `hoàng hoa thám`).

**Cách làm:**
- Chuẩn hoá mỗi giá trị về key so sánh (Unicode NFC + chữ thường).
- Nhóm theo key này.
- Các nhóm có **nhiều hơn 1 biến thể** được xem là trùng do hoa/thường.

**Áp dụng cho các cột:**
- `address`
- `street_name`

##### Kiểm tra

In [9]:
report = audit_case_variants(df, cols)



address: số nhóm khác nhau chỉ do hoa/thường = 0

street_name: số nhóm khác nhau chỉ do hoa/thường = 2


_key
trần văn dư    [Trần Văn Dư, TrầN Văn Dư]
đoàn văn bơ    [Đoàn Văn Bơ, ĐoàN Văn Bơ]
Name: street_name, dtype: object

##### Chuẩn hóa

In [10]:
df[cols] = df[cols].apply(lambda s: s.map(normalize_vn_title))

# Kiểm tra lại
print("Sau khi đã thống nhất lại:")
report_after = audit_case_variants(df, cols)


Sau khi đã thống nhất lại:

address: số nhóm khác nhau chỉ do hoa/thường = 0

street_name: số nhóm khác nhau chỉ do hoa/thường = 0


#### 1.4 Tách quận/huyện từ `address`
Vì bài toán đã quy về địa bàn TP.HCM nên ta chỉ cần giữ quận/huyện.

In [11]:
df["district"] = df["address"].apply(extract_district_from_address)

num_missing = df["district"].isna().sum()
total = len(df)

print(f"Số dòng thiếu district: {num_missing}/{total} ({num_missing/total:.2%})")
df = df.drop(columns=["address"])


Số dòng thiếu district: 0/20912 (0.00%)


### **2. Tạo đặc trưng**

#### 2.1. Trích xuất đặc trưng month (Tháng)

Đặc trưng **month** được trích xuất từ dữ liệu thời gian đăng tin (`date`), giúp chuyển đổi thông tin ngày tháng thô thành một biến số định lượng phản ánh yếu tố thời gian.

**Ý nghĩa của đặc trưng:**
* **Tính thời vụ (Seasonality):** Giá thuê phòng trọ thường biến động theo các thời điểm trong năm (ví dụ: nhu cầu tăng cao vào tháng 8-9 khi sinh viên nhập học).
* **Đơn giản hóa dữ liệu:** Chuyển đổi từ định dạng ngày cụ thể sang số tháng giúp mô hình tập trung vào các quy luật lặp lại hàng năm thay vì bị nhiễu bởi các mốc thời gian quá chi tiết.
* **Tối ưu hóa không gian đặc trưng:** Việc loại bỏ cột `date` gốc sau khi trích xuất giúp giảm bớt các biến không cần thiết cho quá trình huấn luyện mô hình.

**Các bước thực hiện:**
1. Chuyển đổi cột `date` về định dạng `datetime` (xử lý các giá trị lỗi nếu có).
2. Trích xuất giá trị tháng (từ 1 đến 12) vào cột mới đặt tên là `month`.
3. Loại bỏ cột `date` để hoàn thiện bộ dữ liệu đặc trưng.

In [12]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["month"] = df["date"].dt.month
df = df.drop(columns=["date"])


#### 2.2. Đặc trưng Tọa độ địa lý (Latitude & Longitude)

Hai đặc trưng **latitude** và **longitude** được trích xuất từ trường địa chỉ hành chính (`address`) nhằm biểu diễn vị trí phòng trọ dưới dạng tọa độ địa lý đại diện cho khu vực quận/huyện.

##### Ý nghĩa của đặc trưng:
* **Định lượng hóa vị trí:** Chuyển đổi thông tin địa chỉ dạng văn bản sang dạng số, giúp các thuật toán học máy có thể trực tiếp khai thác yếu tố vị trí.
* **Phân tích không gian ở mức khu vực:** Cho phép đánh giá ảnh hưởng của vị trí địa lý (theo quận/huyện, thành phố) đến giá thuê, phù hợp với dữ liệu địa chỉ không đầy đủ hoặc không nhất quán.
* **Cơ sở cho các phân tích mở rộng:** Là nền tảng để thực hiện phân cụm khu vực (clustering), trực quan hóa phân bố phòng trọ trên bản đồ nhiệt (heatmap), hoặc kết hợp với các đặc trưng vị trí khác ở mức vùng.

##### Quy trình trích xuất (Geocoding):
Tọa độ địa lý được chuyển đổi từ địa chỉ văn bản (`address`) thông qua dịch vụ **Geocoding**, với các bước như sau:

1. **Dịch vụ sử dụng:**  
   Áp dụng dịch vụ *Nominatim* thuộc hệ sinh thái *OpenStreetMap* để chuyển đổi địa chỉ hành chính (quận/huyện, thành phố) sang tọa độ địa lý.

2. **Lựa chọn cấp độ địa chỉ (Quận/Huyện):** Quá trình Geocoding được thực hiện dựa trên thông tin **Quận/Huyện** thay vì địa chỉ chi tiết (số nhà, tên đường). Các lý do chính bao gồm:
   * **Tính chính xác của API Nominatim:** Nominatim hoạt động cực kỳ hiệu quả và ổn định với các ranh giới hành chính (administrative boundaries). Ngược lại, dữ liệu tên đường và hẻm nhỏ tại Việt Nam trên hệ thống OpenStreetMap đôi khi chưa được cập nhật đầy đủ, dễ dẫn đến lỗi định vị sai lệch nếu truy vấn quá chi tiết.
   * **Tránh nhiễu từ dữ liệu người dùng:** Địa chỉ chi tiết từ tin đăng thường chứa nhiều thông tin nhiễu (sai chính tả, tên đường tự phát, hoặc mô tả vị trí theo kiểu "gần chợ", "đối diện trường"). Việc tập trung vào cấp Quận/Huyện giúp loại bỏ hoàn toàn các sai số này.
   * **Tối ưu tỷ lệ khớp dữ liệu (Success Rate):** Geocoding cấp hành chính đảm bảo tỷ lệ thành công gần như 100%, tránh việc phát sinh quá nhiều giá trị thiếu (**NaN**) như khi cố gắng tìm kiếm tọa độ của các hẻm nhỏ chưa có trên bản đồ số toàn cầu. Khi dùng tọa độ với đơn vị số nhà hoặc đơn vị tên đường thì Nominatim cho ra kết quả sai thậm chí không trả về kết quả nên việc chọn đơn vị hành chính cấp quận/huyện cho ra được kết quả tốt nhất.

3. **Tối ưu hóa hiệu năng:**  
   * Sử dụng cơ chế **caching** để lưu trữ kết quả geocoding của các địa chỉ trùng lặp, giúp giảm số lượng request gửi đến API và tăng tốc độ xử lý.
   * Thực hiện giới hạn tần suất request nhằm tuân thủ chính sách sử dụng của Nominatim.

4. **Xử lý ngoại lệ:**  
   Những địa chỉ không xác định được tọa độ sẽ được gán giá trị thiếu (**NaN**) và không tham gia vào các phân tích không gian tiếp theo.

##### Lưu ý kỹ thuật:
Tọa độ thu được mang tính **đại diện cho khu vực hành chính** (quận/huyện), không phản ánh chính xác vị trí từng phòng trọ cụ thể. Trong bài toán dự đoán giá thuê theo vùng, cách tiếp cận này cung cấp một "điểm neo" địa lý đủ tin cậy để mô hình học được mối tương quan giữa vị trí khu vực và mức giá trung bình.

In [13]:
CACHE_PATH = "geocode_cache.csv"

# Tạo cột query chuẩn Nominatim
df["address_nominatim"] = df["district"].apply(build_area_nominatim)

# Load cache từ file (nếu có)
geo_cache = load_cache(CACHE_PATH)

# Lấy danh sách unique query cần geocode (chưa có trong cache)
unique_queries = df["address_nominatim"].dropna().astype(str).unique()
to_geocode = [q for q in unique_queries if q not in geo_cache]

print(f"Unique queries: {len(unique_queries)}")
print(f"Need to geocode (not in cache): {len(to_geocode)}")

# Geocode chỉ những query thiếu
geocode_func = setup_geocoder()
tqdm.pandas()

new_rows = []
for q in tqdm(to_geocode, desc="Geocoding unique queries"):
    lat, lon = get_lat_lon(q, geocode_func)
    geo_cache[q] = (lat, lon)
    new_rows.append((q, lat, lon))

# Lưu cache lại (để lần sau chạy không tốn request)
if len(new_rows) > 0:
    save_cache(geo_cache, CACHE_PATH)
    print(f"Saved cache: {CACHE_PATH} (+{len(new_rows)} new)")
else:
    print("No new queries to geocode. Cache already covers all.")

# Map tọa độ ngược lại cho toàn bộ df (NHANH)
df["latitude"]  = df["address_nominatim"].map(lambda q: geo_cache.get(q, (np.nan, np.nan))[0])
df["longitude"] = df["address_nominatim"].map(lambda q: geo_cache.get(q, (np.nan, np.nan))[1])

print(f"Done! NaN latitude: {df['latitude'].isna().sum()}")


Unique queries: 24
Need to geocode (not in cache): 24


Geocoding unique queries: 100%|██████████| 24/24 [00:32<00:00,  1.34s/it]

Saved cache: geocode_cache.csv (+24 new)
Done! NaN latitude: 0


##### Kiểm tra

In [14]:
print("Ví dụ một số sample sau khi lấy tọa độ:")
display(
    df.loc[df["latitude"].notna(),
           ["street_name", "district", "address_nominatim", "latitude", "longitude"]]
      .head(10)
)


Ví dụ một số sample sau khi lấy tọa độ:


,street_name,district,address_nominatim,latitude,longitude
0,Phạm Hùng,Quận 8,"Quận 8, Thành phố Hồ Chí Minh, Việt Nam",10.721116,106.629208
1,Nguyễn Lương Bằng,Quận 7,"Quận 7, Thành phố Hồ Chí Minh, Việt Nam",10.737567,106.729660
2,Thành Thái,Quận 10,"Quận 10, Thành phố Hồ Chí Minh, Việt Nam",10.772528,106.668202
3,Số 8,Thủ Đức,"Thành phố Thủ Đức, Thành phố Hồ Chí Minh, Việt...",10.851024,106.754895
4,Phạm Ngũ Lão,Quận 1,"Quận 1, Thành phố Hồ Chí Minh, Việt Nam",10.775394,106.699625
5,Trần Văn Đang,Quận 3,"Quận 3, Thành phố Hồ Chí Minh, Việt Nam",10.778891,106.686997
6,Lê Văn Chí,Thủ Đức,"Thành phố Thủ Đức, Thành phố Hồ Chí Minh, Việt...",10.851024,106.754895
8,Số 2,Quận 8,"Quận 8, Thành phố Hồ Chí Minh, Việt Nam",10.721116,106.629208
9,Hoàng Hoa Thám,Quận Tân Bình,"Quận Tân Bình, Thành phố Hồ Chí Minh, Việt Nam",10.803093,106.652352
10,D2,Quận Bình Thạnh,"Quận Bình Thạnh, Thành phố Hồ Chí Minh, Việt Nam",10.812271,106.704039


##### Xóa các thuộc tính hỗ trợ tạo feature mới

In [15]:
cols_to_drop = [
    "address_nominatim"
]

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])


#### 2.3. Đặc trưng Khoảng cách đến trung tâm (Distance to CBD)

Đặc trưng **dist_to_q1_km** được tạo ra nhằm định lượng hóa sự cách biệt về địa lý giữa vị trí phòng trọ và "lõi" trung tâm của Thành phố Hồ Chí Minh (Quận 1).

##### Ý nghĩa của đặc trưng:
* **Phản ánh giá trị kinh tế:** Trong thị trường bất động sản và cho thuê, khoảng cách tới trung tâm thường tỉ lệ nghịch với giá thuê. Đây là biến số quan trọng giúp mô hình học máy bắt được quy luật về vị trí đắc địa.
* **Đại diện cho tiện ích:** Khoảng cách này gián tiếp thể hiện khả năng tiếp cận các dịch vụ cao cấp, khu vui chơi giải trí và các tòa nhà văn phòng tập trung tại khu vực lõi.
* **Chuẩn hóa đơn vị đo lường:** Thay vì chỉ sử dụng tọa độ (vĩ độ, kinh độ) mang tính định danh vị trí, việc chuyển sang đơn vị kilomet (km) giúp mô hình dễ dàng học được mối quan hệ tuyến tính hoặc phi tuyến với biến mục tiêu (giá phòng).

##### Quy trình tính toán:

1. **Xác định tọa độ tham chiếu (Reference Point):**
   Sử dụng tọa độ của Quận 1 làm điểm gốc ($Q1_{lat}$, $Q1_{lon}$) thông qua truy vấn Nominatim: `"Quận 1, Thành phố Hồ Chí Minh, Việt Nam"`.

2. **Công thức tính toán (Haversine Formula):**
   Do bề mặt Trái Đất là hình cầu, khoảng cách giữa hai điểm tọa độ được tính toán bằng công thức Haversine để đảm bảo độ chính xác trên mặt cầu thay vì khoảng cách Euclidean phẳng:

   $$d = 2r \arcsin\left(\sqrt{\sin^2\left(\frac{\phi_2 - \phi_1}{2}\right) + \cos(\phi_1) \cos(\phi_2) \sin^2\left(\frac{\lambda_2 - \lambda_1}{2}\right)}\right)$$

   *Trong đó:*
   * $\phi_1, \phi_2$: Vĩ độ của điểm 1 và điểm 2 (tính bằng radian).
   * $\lambda_1, \lambda_2$: Kinh độ của điểm 1 và điểm 2 (tính bằng radian).
   * $r$: Bán kính trung bình của Trái Đất ($\approx 6371$ km).



3. **Xử lý dữ liệu hàng loạt:**
   * **Lọc dữ liệu:** Chỉ thực hiện tính toán trên các bản ghi có dữ liệu tọa độ hợp lệ (không phải **NaN**).
   * **Vectorization:** Sử dụng thư viện NumPy để thực hiện tính toán vector hóa trên toàn bộ cột dữ liệu, giúp tối ưu hóa hiệu năng so với việc sử dụng vòng lặp truyền thống.

In [16]:
Q1_QUERY = "Quận 1, Thành phố Hồ Chí Minh, Việt Nam"
Q1_LAT, Q1_LON = get_lat_lon(Q1_QUERY, geocode_func)

print("Q1 location:", Q1_LAT, Q1_LON)


Q1 location: 10.7753939 106.6996249


In [17]:
mask = df["latitude"].notna() & df["longitude"].notna()
df["dist_to_q1_km"] = np.nan

df.loc[mask, "dist_to_q1_km"] = haversine_km(
    df.loc[mask, "latitude"].to_numpy(),
    df.loc[mask, "longitude"].to_numpy(),
    Q1_LAT,
    Q1_LON
)


##### Preview

In [18]:
print("Ví dụ sample sau khi FE dist_to_q1_km:")
display(
    df.loc[df["dist_to_q1_km"].notna(),
           ["district", "latitude", "longitude", "dist_to_q1_km"]]
      .head(5)
)

print("Thống kê dist_to_q1_km:")
display(df["dist_to_q1_km"].describe())
print("Location xa Quận 1 nhất:")
display(
    df.loc[df["dist_to_q1_km"].notna(),
           ["district", "latitude", "longitude", "dist_to_q1_km"]]
      .sort_values("dist_to_q1_km", ascending=False)
      .head(2)
)


Ví dụ sample sau khi FE dist_to_q1_km:


,district,latitude,longitude,dist_to_q1_km
0,Quận 8,10.721116,106.629208,9.777700
1,Quận 7,10.737567,106.729660,5.334517
2,Quận 10,10.772528,106.668202,3.447223
3,Thủ Đức,10.851024,106.754895,10.351990
4,Quận 1,10.775394,106.699625,0.000000


Thống kê dist_to_q1_km:


count    20912.000000
mean         6.581468
std          3.242042
min          0.000000
25%          4.128798
50%          6.012419
75%          8.061707
max         47.921041
Name: dist_to_q1_km, dtype: float64

Location xa Quận 1 nhất:


,district,latitude,longitude,dist_to_q1_km
16145,Huyện Cần Giờ,10.396588,106.908698,47.921041
6992,Huyện Củ Chi,11.096707,106.509746,41.306853


#### 2.4. Đặc trưng Giá thuê Trung vị theo Quận/Huyện (District Median Price)

Đặc trưng **district_median_price** là một dạng biến số hóa dựa trên mục tiêu (Target Encoding), đại diện cho mức giá thuê phổ biến tại từng đơn vị hành chính cấp Quận/Huyện.

##### Ý nghĩa của đặc trưng:
* **Phản ánh mặt bằng giá khu vực:** Giúp mô hình nắm bắt được "điểm chuẩn" (benchmark) về giá của từng quận. Ví dụ: Giá trung vị tại Quận 1 sẽ mặc định cao hơn so với Quận Bình Tân.
* **Tính ổn định (Robustness):** Sử dụng giá trị **trung vị (Median)** thay vì trung bình cộng (Mean) giúp đặc trưng này không bị lệch bởi các điểm dữ liệu cực biên (outliers) như các phòng trọ cao cấp hoặc lỗi nhập liệu giá quá thấp.
* **Giảm chiều dữ liệu:** Thay vì sử dụng biến phân loại (Categorical) với hàng chục nhãn quận/huyện khác nhau (có thể gây hiện tượng cực thưa dữ liệu - sparsity), việc chuyển sang giá trị số giúp mô hình học máy xử lý hiệu quả hơn.

##### Quy trình trích xuất:

1. **Chuẩn hóa dữ liệu đầu vào:**
   * Chuyển đổi cột giá thuê (`price`) về dạng số (numeric), xử lý các giá trị không hợp lệ thành **NaN**.
   * Làm sạch tên quận/huyện (`district`) để đảm bảo không có sự sai lệch giữa các nhãn tương đồng (ví dụ: "Quận 1" và "Q.1").

2. **Tính toán giá trị đại diện:**
   * Gom nhóm dữ liệu (`groupby`) theo từng Quận/Huyện.
   * Tính toán giá trị trung vị của cột giá trong từng nhóm.

3. **Ánh xạ (Mapping):**
   * Ánh xạ các giá trị trung vị vừa tính được ngược lại tập dữ liệu gốc dựa trên tên quận tương ứng, tạo thành một đặc trưng số mới tương quan trực tiếp với biến mục tiêu.

##### Thực hiện

In [19]:
df = add_district_median_price(
    df,
    price_col="price",
    district_col="district",
    out_col="district_median_price"
)

df[["district", "price", "district_median_price"]].head(5)


,district,price,district_median_price
0,Quận 8,2.8,3.8
1,Quận 7,1.3,4.0
2,Quận 10,4.5,4.5
3,Thủ Đức,1.2,3.0
4,Quận 1,1.5,5.0


#### 2.5. Đặc trưng Tuyến đường Nổi bật (Premium Hot Street Flag)

Đặc trưng **is_hot_street** là một biến nhị phân (Binary Feature) được xác định dựa trên sự kết hợp giữa **mật độ tin đăng lớn** và **giá trị vượt trội** của tuyến đường đó so với mặt bằng chung của Quận/Huyện.

##### Ý nghĩa của đặc trưng:
* **Xác định các "Điểm nóng" giá trị:** Thay vì chỉ đếm số lượng tin đăng, đặc trưng này lọc ra các tuyến đường vừa có tính thanh khoản cao (nhiều tin đăng), vừa có mức giá "premium" (cao cấp) hơn so với khu vực lân cận.
* **Chuẩn hóa theo khu vực (Local Normalization):** Việc so sánh với trung vị của Quận (`district_median_price`) giúp loại bỏ sự chênh lệch giá giữa các Quận. Một con đường được coi là "Hot" khi nó đắt đỏ vượt trội so với các con đường khác trong cùng một Quận, bất kể giá tuyệt đối của nó cao hay thấp so với trung tâm thành phố.
* **Đảm bảo tính đại diện thống kê:** Việc áp dụng ngưỡng tối thiểu 15 tin đăng giúp loại bỏ các trường hợp giá trung vị bị lệch do số lượng mẫu quá ít hoặc do các tin đăng ảo/ngoại lai.

##### Quy trình trích xuất:

1. **Thống kê theo cặp (Quận - Đường):**
   Tiến hành gom nhóm dữ liệu theo đồng thời `district` và `street_name` để tính toán chính xác số lượng tin đăng và giá trị đặc trưng cho từng tuyến đường cụ thể trong mỗi đơn vị hành chính.

2. **Lọc theo mật độ tin đăng (Volume Filter):**
   Chỉ những tuyến đường có số lượng mẫu ($count_{street}$) đạt ngưỡng tối thiểu mới được đưa vào tính toán giá trị trung vị:
   $$count_{street} \ge 15$$

3. **Tính toán tỷ lệ giá tương đối (Relative Price Ratio):**
   Xác định vị thế giá của tuyến đường bằng cách so sánh với mức giá trung vị của toàn Quận:
   $$street\_relative\_price = \frac{Street\_Median\_Price}{District\_Median\_Price}$$

4. **Gán nhãn Tuyến đường Nổi bật (Labeling):**
   Cờ `is_hot_street` được thiết lập dựa trên ngưỡng tỷ lệ giá (mặc định là 1.2, tức đắt hơn 20% so với Quận):
   $$is\_hot\_street = \begin{cases} 1 & \text{nếu } street\_relative\_price \ge 1.2 \\ 0 & \text{ngược lại} \end{cases}$$

##### Lưu ý về tham số:
* **Ngưỡng mẫu (`min_ads_street = 15`):** Đảm bảo giá trị trung vị (`median`) có độ tin cậy cao về mặt thống kê.
* **Ngưỡng cao cấp (`premium_ratio = 1.15`):** Đây là tham số linh hoạt giúp mô hình tập trung vào nhóm các tuyến đường thuộc phân khúc cao cấp nhất của địa phương.
* **Xử lý giá trị thiếu:** Những con đường không

In [20]:
df = add_hot_street_by_relative_price(df, premium_ratio=1.15, min_ads_street=15)

df.loc[df["is_hot_street"] == 1, [
    "district",
    "street_name",
    "street_median_price",
    "district_median_price",
    "street_relative_price"
]].drop_duplicates("street_name").sort_values(
    "street_relative_price", ascending=False
).head(5)


,district,street_name,street_median_price,district_median_price,street_relative_price
949,Quận 9,Tây Hòa,4.50,3.3,1.363636
339,Quận 12,Hà Thị Khiêm,3.90,3.0,1.300000
1326,Quận 5,Nguyễn Trãi,4.90,3.8,1.289474
3557,Quận Tân Bình,Yên Thế,5.15,4.0,1.287500
1345,Thủ Đức,Hoàng Diệu,3.80,3.0,1.266667


##### Xem phân bố

In [21]:
df["is_hot_street"].value_counts(normalize=True)


is_hot_street
0    0.962175
1    0.037825
Name: proportion, dtype: float64

##### Xóa các thuộc tính hỗ trợ tạo feature mới

In [22]:
cols_to_drop = [
    "street_median_price",
    "street_relative_price",
    "street_ads_count",
]

df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])


#### 2.6. Chỉ số Tiện ích Tích hợp (Amenity Score)

Đặc trưng **amenity_ratio** được xây dựng bằng cách tổng hợp 11 đặc trưng nhị phân đại diện cho các tiện nghi hiện có trong phòng trọ (Máy lạnh, tủ lạnh, thang máy, bãi đậu xe,...).

##### Ý nghĩa của đặc trưng:
* **Định lượng mức độ tiện nghi:** Chuyển đổi trạng thái "có/không" của các tiện ích rời rạc thành một thang đo liên tục từ 0 đến 1, đại diện cho mức độ trang bị đầy đủ của phòng trọ.
* **Đại diện cho phân khúc phòng:** Chỉ số này giúp phân loại nhanh các phòng trọ từ phân khúc "Phòng trống/Cơ bản" (ratio thấp) đến "Căn hộ dịch vụ Full nội thất" (ratio cao).
* **Tối ưu hóa biến số:** Giúp mô hình học máy nhận diện tổng quan về giá trị gia tăng của gói tiện ích thay vì phải xử lý quá nhiều biến nhị phân thưa thớt.

##### Quy trình trích xuất:
1. **Số hóa và Làm sạch:** Toàn bộ các cột tiện ích được chuyển đổi sang dạng số (0 hoặc 1). Các giá trị thiếu (**NaN**) được xử lý là 0 (không có tiện ích).
2. **Tính toán chỉ số trung bình:**
   $$amenity\_ratio = \frac{\sum_{i=1}^{n} Amenity_i}{n}$$
   *Trong đó $n=11$ là tổng số loại tiện ích được khảo sát.*

In [23]:
amenity_cols = [
    "air_conditioning",
    "fridge",
    "washing_machine",
    "mezzanine",
    "kitchen",
    "wardrobe",
    "bed",
    "balcony",
    "elevator",
    "free_time",
    "parking",
]

# tỷ lệ tiện ích có / tổng tiện ích
df["amenity_ratio"] = df[amenity_cols].mean(axis=1)

df[amenity_cols + ["amenity_ratio"]].head(5)


,air_conditioning,fridge,washing_machine,mezzanine,kitchen,wardrobe,bed,balcony,elevator,free_time,parking,amenity_ratio
0,0,0,0,0,1,0,0,0,0,0,0,0.090909
1,0,1,1,0,1,0,0,0,0,0,0,0.272727
2,0,0,0,0,0,0,0,0,0,1,0,0.090909
3,1,1,1,0,1,1,1,0,0,0,0,0.545455
4,1,0,0,0,1,0,1,0,0,1,0,0.363636


## **Xử lí Missing value**

##### **Kiểm tra missing**

In [24]:
missing_cnt = df.isna().sum()
missing_pct = (missing_cnt / len(df)) * 100

missing_df = (
    pd.DataFrame({
        "missing_count": missing_cnt,
        "missing_percent": missing_pct
    })
    .query("missing_count > 0")
    .sort_values("missing_percent", ascending=False)
)

if missing_df.empty:
    print("Không có giá trị thiếu (No missing values)")
else:
    display(missing_df)


Không có giá trị thiếu (No missing values)


**Bộ dữ liệu không ghi nhận giá trị thiếu (missing values) do các nguyên nhân sau:**

* **Quy định đăng tin nghiêm ngặt:** Website nguồn áp dụng các quy tắc bắt buộc người dùng phải cung cấp đầy đủ thông tin (địa chỉ, giá tiền, diện tích) để đảm bảo tính minh bạch và uy tín cho tin đăng.
* **Xác thực từ hệ thống:** Nhờ cơ chế kiểm soát đầu vào của website, các tin đăng thiếu thông tin quan trọng sẽ không được hiển thị, giúp dữ liệu khi crawl về đã được "lọc sạch" và hoàn thiện ngay từ đầu.

## **Xử lí Outliers**
Quy trình thực hiện giới hạn các giá trị ngoại lai bằng phương pháp **IQR Capping** để đảm bảo tính ổn định cho dữ liệu.

#### 1. Trước khi xử lý outliers
Xác định danh sách các cột mục tiêu và sao lưu dữ liệu trước khi thực hiện thay đổi.

In [25]:
outlier_cols = [
    "area",
    "dist_to_q1_km"
]

df_before_outlier = df[outlier_cols].copy()
display(df[outlier_cols].describe())


,area,dist_to_q1_km
count,20912.000000,20912.000000
mean,28.123201,6.581468
std,14.544268,3.242042
min,1.000000,0.000000
25%,22.000000,4.128798
50%,25.000000,6.012419
75%,30.000000,8.061707
max,300.000000,47.921041


#### 2. IQR capping
Chúng ta sử dụng hàm `iqr_cap` để xử lý các giá trị ngoại lai. Thay vì loại bỏ hoàn toàn các dòng dữ liệu, chúng ta thực hiện "ghìm" (capping) các giá trị này về các ngưỡng biên an toàn.

##### 2.1. Công thức toán học
Hàm dựa trên quy tắc dải biến phân nội phần vị (Interquartile Range - IQR):
* **Dải IQR**: $IQR = Q_3 - Q_1$
* **Ngưỡng dưới (Lower Bound)**: $L = Q_1 - k \times IQR$
* **Ngưỡng trên (Upper Bound)**: $U = Q_3 + k \times IQR$

Trong đó $Q_1$ và $Q_3$ lần lượt là tứ phân vị thứ nhất (25%) và thứ ba (75%). Tham số $k$ thường được chọn là **1.5**.

##### 2.2. Lí do chọn phương pháp này
* **Bảo toàn kích thước tập dữ liệu**: Không làm mất các quan sát quan trọng trong các cột khác của cùng một dòng.
* **Tính bền vững (Robustness)**: Tứ phân vị không bị ảnh hưởng bởi chính các giá trị ngoại lai, khác với phương pháp Z-score (dựa trên Mean và Std - vốn rất nhạy cảm với outlier).
* **Kiểm soát nhiễu**: Giúp các mô hình học máy (đặc biệt là các mô hình dựa trên khoảng cách như Linear Regression, KNN) không bị chệch hướng bởi các giá trị quá lớn hoặc quá nhỏ.

In [26]:
for col in outlier_cols:
    df[col] = iqr_cap(df[col])

for col in outlier_cols:
    capped = (df_before_outlier[col] != df[col]).sum()
    print(f"{col}: {capped} giá trị bị capping.")


area: 1023 giá trị bị capping.
dist_to_q1_km: 412 giá trị bị capping.


#### 3. So sánh trước và sau khi xử lí outliers
Kiểm tra hiệu quả của việc xử lý bằng cách đếm số lượng giá trị đã bị thay đổi (capped) ở mỗi cột.

In [27]:
compare_df = pd.DataFrame({
    "min_before": df_before_outlier.min(),
    "max_before": df_before_outlier.max(),
    "min_after": df[outlier_cols].min(),
    "max_after": df[outlier_cols].max(),
})

display(compare_df)


,min_before,max_before,min_after,max_after
area,1.0,300.000000,10.0,42.000000
dist_to_q1_km,0.0,47.921041,0.0,13.961071


## **Ghi ra file**

#### 1. Review lại

In [28]:
print("Các feature được sử dụng:")
print(df.columns.tolist())


Các feature được sử dụng:
['street_name', 'price', 'area', 'air_conditioning', 'fridge', 'washing_machine', 'mezzanine', 'kitchen', 'wardrobe', 'bed', 'balcony', 'elevator', 'free_time', 'parking', 'district', 'month', 'latitude', 'longitude', 'dist_to_q1_km', 'district_median_price', 'is_hot_street', 'amenity_ratio']


##### 2. Ghi ra file

In [29]:
output_path = "../data/processed.csv"

df.to_csv(
    output_path,
    index=False,
    encoding="utf-8-sig"
)

print("Saved to:", output_path)


Saved to: ../data/processed.csv


##### 3. Tổng kết quy trình xử lý dữ liệu

* **Khu trú phạm vi:** Tập trung dữ liệu vào bài toán dự đoán giá thuê tại địa bàn TP.HCM.
* **Tinh gọn dữ liệu:** Loại bỏ các cột nhiễu không mang giá trị dự báo cho mô hình bao gồm `url`, `description`, `title`, và `location`.
* **Làm sạch giá trị biên:** Loại bỏ các bản ghi có giá thuê (`price`) và diện tích (`area`) phi thực tế để loại trừ dữ liệu rác.
* **Chuẩn hóa văn bản:** Áp dụng chuẩn Unicode NFC, xử lý khoảng trắng và đồng nhất định dạng chữ hoa/thường để loại bỏ các bản ghi trùng lặp về nội dung.
* **Kỹ thuật đặc trưng thời gian:** Xử lý dữ liệu cột `date` và trích xuất thành phần tháng (`month`) để bắt kịp yếu tố thời vụ.
* **Xử lý đơn vị hành chính:** Phân tách quận/huyện (`district`) từ địa chỉ và thực hiện kiểm tra nghiêm ngặt các giá trị thiếu.
* **Làm giàu dữ liệu không gian:** * Thực hiện Geocode tọa độ theo quận/huyện với cơ chế cache nhằm tối ưu hóa hiệu suất gọi API Nominatim.
    * Tính toán khoảng cách đến trung tâm (`dist_to_q1_km`) để đưa yếu tố vị trí địa lý vào mô hình.
* **Xây dựng đặc trưng khu vực:** Tạo các biến thống kê mức giá theo khu vực và tuyến đường như `district_median_price`, `street_median_price`, `street_relative_price` và nhận diện các tuyến đường "hot" (`is_hot_street`).
* **Kiểm soát ngoại lai:** Xử lý các giá trị đột biến (outliers) bằng phương pháp IQR capping đối với tất cả các biến số liên tục để tăng độ ổn định cho mô hình.